In [ ]:
# Homework 1:

# Using chinook.db write pandas code.

import sqlite3
import pandas as pd


# Подключение к базе данных
conn = sqlite3.connect("chinook.db")

# Загружаем нужные таблицы
customers = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql("SELECT InvoiceId, CustomerId, Total FROM invoices", conn)
invoice_items = pd.read_sql_query("SELECT * FROM invoice_items", conn)
tracks = pd.read_sql_query("SELECT * FROM tracks", conn)
albums = pd.read_sql_query("SELECT * FROM albums", conn)


# 1. Customer Purchases Analysis:

# - Find the total amount spent by each customer on purchases (considering invoices).
# - Identify the top 5 customers with the highest total purchase amounts.
# - Display the customer ID, name, and the total amount spent for the top 5 customers.


# Считаем сумму покупок по каждому инвойсу
invoice_totals = invoice_items.groupby('InvoiceId')['UnitPrice'].sum().reset_index()

# Присоединяем суммы к инвойсам и клиентам
invoices_with_totals = invoices.merge(invoice_totals, on='InvoiceId')
customer_spending = invoices_with_totals.groupby(['CustomerId']).agg({'UnitPrice':'sum'}).reset_index()
customer_spending = customer_spending.merge(customers[['CustomerId', 'FirstName', 'LastName']], on='CustomerId')

# Переименуем столбцы для наглядности
customer_spending.rename(columns={'UnitPrice':'TotalSpent'}, inplace=True)

# Топ-5 клиентов
top5_customers = customer_spending.sort_values(by='TotalSpent', ascending=False).head(5)
print(top5_customers[['CustomerId', 'FirstName', 'LastName', 'TotalSpent']])




    CustomerId FirstName    LastName  TotalSpent
5            6    Helena        Holý       49.62
25          26   Richard  Cunningham       47.62
56          57      Luis       Rojas       46.62
44          45  Ladislav      Kovács       45.62
45          46      Hugh    O'Reilly       45.62


In [ ]:
# 2. Album vs. Individual Track Purchases:

# - Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
# - A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
# - Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

In [3]:
import sqlite3
import pandas as pd

# Подключение к базе
conn = sqlite3.connect("chinook.db")

# Загружаем нужные таблицы
customers = pd.read_sql_query("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM invoices", conn)
invoice_items = pd.read_sql_query("SELECT InvoiceId, TrackId FROM invoice_items", conn)
tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM tracks", conn)

# 1) Считаем количество треков в каждом альбоме
album_counts = tracks.groupby('AlbumId').TrackId.nunique().reset_index()
album_counts.rename(columns={'TrackId':'TotalTracks'}, inplace=True)

# 2) Сколько треков каждого альбома купил каждый клиент
cust_album = (invoice_items
              .merge(invoices, on="InvoiceId")
              .merge(tracks, on="TrackId")
              .groupby(['CustomerId','AlbumId']).TrackId.nunique().reset_index()
              .rename(columns={'TrackId':'TracksBought'}))

# 3) Определяем: купил ли клиент полный альбом
cust_album = cust_album.merge(album_counts, on="AlbumId")
cust_album['FullAlbum'] = cust_album['TracksBought'] == cust_album['TotalTracks']

# 4) Для каждого клиента проверяем: есть ли хотя бы один полный альбом
cust_type = cust_album.groupby('CustomerId')['FullAlbum'].any().reset_index()
cust_type['PurchaseType'] = cust_type['FullAlbum'].map({True:'Full Album', False:'Individual Tracks Only'})

# 5) Соединяем с именами
cust_type = cust_type.merge(customers, on='CustomerId')

# 🔹 Итоговый список клиентов
print(cust_type[['FirstName','LastName','PurchaseType']])

# 🔹 Процент клиентов по категориям
summary = cust_type['PurchaseType'].value_counts(normalize=True) * 100
print("\nPercentage of customers by purchase type:")
print(summary)


    FirstName      LastName            PurchaseType
0        Luís     Gonçalves              Full Album
1      Leonie        Köhler              Full Album
2    François      Tremblay              Full Album
3       Bjørn        Hansen              Full Album
4   František   Wichterlová  Individual Tracks Only
5      Helena          Holý  Individual Tracks Only
6      Astrid        Gruber              Full Album
7        Daan       Peeters  Individual Tracks Only
8        Kara       Nielsen  Individual Tracks Only
9     Eduardo       Martins  Individual Tracks Only
10  Alexandre         Rocha  Individual Tracks Only
11    Roberto       Almeida  Individual Tracks Only
12   Fernanda         Ramos              Full Album
13       Mark       Philips  Individual Tracks Only
14   Jennifer      Peterson  Individual Tracks Only
15      Frank        Harris  Individual Tracks Only
16       Jack         Smith  Individual Tracks Only
17   Michelle        Brooks  Individual Tracks Only
18        Ti